In [1]:
%pip install -q PyAthena[SQLAlchemy]

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q missingno

Note: you may need to restart the kernel to use updated packages.


In [236]:
%pip install -q imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


## Prepare Datasets for Predictor Training, Validation and Testing

#### TS

Import modules that build patient cohort, extract demographics and lab events data:

In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from matplotlib import pyplot
%matplotlib inline
import missingno as msno
from sklearn.preprocessing import OneHotEncoder

In [50]:
# from dataproc.cohort import query_esbl_pts, remove_dups, observation_window
from dataproc import cohort
from dataproc import create_dataset
from dataproc.sampling import generate_samples

from hyper_params import HyperParams

In [51]:
# load hyperparams instance
params = HyperParams()

In [52]:
DATA_DIR='data'

In [53]:
def write_dataframe(df, filename, data_dir = DATA_DIR):
    destination = f"{DATA_DIR}/{filename}.parquet"
    df.to_parquet(destination)
    
    
def load_dataframe(filename, data_dir = DATA_DIR):
    destination = f"{DATA_DIR}/{filename}.parquet"
    df = pd.read_parquet(destination)
    return df


In [54]:
def drop_sparse_columns(df, columns:list, max_sparsity_to_keep=0.9):
    # count the missing values in each column
    missing_values_counts = df[columns].isna().sum()
    # percent of columns with rare values
    max_missing_vals = max_sparsity_to_keep * df.shape[0]
    sparse_columns = missing_values_counts[missing_values_counts > max_missing_vals].index.tolist()
    # drop the rare labtests from the dataset
    df = df.drop(columns=sparse_columns)
    return df




## create list of patients, max_observation_window

- create list of elibible patients
	- did not die during observation_window
	- were not discharbed during observation window
- calculate index_time

In [8]:
df_all_pts_within_observation_window, view_name_all_pts_within_observation_window = cohort.query_all_pts_within_observation_window(params.observation_window_hours)

In [11]:
view_name_all_pts_within_observation_window, df_all_pts_within_observation_window.shape

('default.all_pts_6_hours', (58397, 3))

In [12]:
df_all_pts_within_observation_window.head()

,subject_id,hadm_id,index_date
0,50488,142899,2183-01-07 00:49:00
1,19548,193954,2170-05-10 13:15:00
2,84461,146684,2177-02-20 03:03:00
3,1750,131278,2167-09-24 17:04:00
4,30582,187871,2129-04-07 18:00:00


In [15]:
write_dataframe(df_all_pts_within_observation_window, 'df_all_pts_within_observation_window')

## @

In [8]:
# load prev results
view_name_all_pts_within_observation_window = 'default.all_pts_6_hours' 
df_all_pts_within_observation_window = load_dataframe('df_all_pts_within_observation_window')

## generate features for all patients (under observation window)

## Static features
	- save static feature names

In [18]:
df_static_data = create_dataset.static_data(hadm_ids_table = view_name_all_pts_within_observation_window)
df_static_data = df_static_data.drop(columns=['admittime'])

In [67]:
static_feature_names = df_static_data.columns.tolist()

In [68]:
static_feature_names

['hadm_id',
 'admission_type',
 'admission_location',
 'insurance',
 'language',
 'religion',
 'marital_status',
 'ethnicity',
 'gender',
 'age']

In [21]:
df_static_data.head()

,hadm_id,admittime,admission_type,admission_location,insurance,language,religion,marital_status,ethnicity,gender,age
0,165315,2196-04-09 12:26:00,EMERGENCY,EMERGENCY ROOM ADMIT,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,F,64
1,152223,2153-09-03 07:15:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Medicare,NaN,CATHOLIC,MARRIED,WHITE,M,71
2,124321,2157-10-18 19:34:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,M,75
3,161859,2139-06-06 16:14:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,M,39
4,129635,2160-11-02 02:06:00,EMERGENCY,EMERGENCY ROOM ADMIT,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,M,58


In [65]:
df_static_data.shape

(58397, 11)

In [72]:
def process_static_data(dataset):
    dataset['admission_location'] = \
    np.where(dataset['admission_location'].isin(['** INFO NOT AVAILABLE **']), 'EMERGENCY ROOM ADMIT',
    np.where(dataset['admission_location'].isin(['TRANSFER FROM SKILLED NUR','TRANSFER FROM OTHER HEALT',
                            'TRANSFER FROM HOSP/EXTRAM']), 'TRANSFER FROM MED FACILITY',dataset['admission_location']))
    dataset['language'] = \
    np.where(~dataset['language'].isin(['ENGL','SPAN']),'OTHER',dataset['language'])

    dataset['religion'] = \
    np.where(~dataset['religion'].isin(['CATHOLIC','NOT SPECIFIED','UNOBTAINABLE','PROTESTANT QUAKER','JEWISH']),'OTHER',
    np.where(dataset['religion'].isin(['UNOBTAINABLE']),'NOT SPECIFIED', dataset['religion'] ))

    dataset['ethnicity'] = \
    np.where(dataset['ethnicity'].isin(['ASIAN - CHINESE',
                                        'ASIAN - ASIAN INDIAN',
                                        'ASIAN - VIETNAMESE',
                                        'ASIAN - OTHER',
                                        'ASIAN - FILIPINO',
                                        'ASIAN - CAMBODIAN']), 'ASIAN',
    np.where(dataset['ethnicity'].isin(['WHITE - RUSSIAN',
                                        'WHITE - BRAZILIAN',
                                        'WHITE - OTHER EUROPEAN']),'WHITE',
    np.where(dataset['ethnicity'].isin(['BLACK/CAPE VERDEAN',
                                        'BLACK/HAITIAN',
                                        'BLACK/AFRICAN']), 'BLACK/AFRICAN AMERICAN',
    np.where(dataset['ethnicity'].isin(['HISPANIC/LATINO - PUERTO RICAN',
                                       'HISPANIC/LATINO - DOMINICAN',
                                       'HISPANIC/LATINO - SALVADORAN',
                                       'HISPANIC/LATINO - CUBAN',
                                       'HISPANIC/LATINO - MEXICAN']), 'HISPANIC OR LATINO',   
    np.where(dataset['ethnicity'].isin(['MULTI RACE ETHNICITY',
                                        'MIDDLE EASTERN',
                                        'PORTUGUESE',
                                        'AMERICAN INDIAN/ALASKA NATIVE',
                                        'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
                                        'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE']), 'OTHER',
    np.where(dataset['ethnicity'].isin(['UNABLE TO OBTAIN',
                                        'PATIENT DECLINED TO ANSWER']), 'UNKNOWN/NOT SPECIFIED',
    dataset['ethnicity']))))))
    dataset['marital_status'] = dataset['marital_status'].fillna(value='UNKNOWN')
    # clean the age column
    threshold = 105
    df_static_data['age'] = df_static_data['age'].apply(lambda x: x if x < threshold else np.nan)
    

In [73]:
process_static_data(df_static_data)

In [74]:
write_dataframe(df_static_data, 'df_static_data')

## @

In [117]:
# load prev results
df_static_data = load_dataframe('df_static_data')
static_feature_names = df_static_data.columns.tolist()

### Examine missing values

### lab results
- no cleaning now!
- save lab features names

In [25]:
df_lab_events = create_dataset.lab_events(view_name_all_pts_within_observation_window)

In [26]:
df_lab_events = df_lab_events.dropna(subset=['value'])

In [27]:
df_lab_events.head()

,hadm_id,itemid,value,flag,charttime
0,193313,50912,2.0,abnormal,2153-12-10 22:40:00
1,193313,50960,1.8,NaN,2153-12-10 22:40:00
2,193313,51146,0.1,NaN,2153-12-10 22:40:00
3,193313,51237,1.2,abnormal,2153-12-10 22:40:00
4,193313,51250,89.0,NaN,2153-12-10 22:40:00


In [50]:
df_lab_events.flag.value_counts()

abnormal    998353
delta         7472
Name: flag, dtype: int64

In [55]:
df_lab_events['flag'].fillna('False').map({'abnormal':True, 'delta':True, 'False': False}).value_counts()

False    2220908
True     1005825
Name: flag, dtype: int64

In [28]:
df_lab_events.shape

(3226733, 5)

In [29]:
write_dataframe(df_lab_events, 'df_lab_events')

## @

In [10]:
# load prev results
df_lab_events = load_dataframe('df_lab_events')
# static_feature_names = df_static_data.columns.tolist()

In [11]:
def keep_last_labtest_instance(df):
    """
    select last instance of every type of test for a patient
    """
    df = df.sort_values('charttime', axis=0)
    df = df.drop_duplicates(subset=['hadm_id','itemid'], keep='last', ignore_index=True)
    return df
    

def pivot_labtests_to_columns(df):
    df = df.reset_index(drop=True)
    df = df.pivot(index=['hadm_id'],columns=['itemid'],values=['value'])
    df.columns = df.columns.to_flat_index()
    df.columns = [str(colname[1]) for colname in df.columns]
    df = df.reset_index(['hadm_id'])

    return df

        

def pivot_flags_to_columns(df):
    df = df.copy()
    df['flag'] = df['flag'].fillna('False').map({'abnormal':1, 'delta':1, 'False': -1})
    df['flag_name'] = df['itemid'].astype(str) + pd.Series(["_flag"] * df.shape[0]).astype(str)
    df = df.pivot(index=['hadm_id'],columns=['flag_name'],values=['flag'])
    df.columns = df.columns.to_flat_index()
    df.columns = [str(colname[1]) for colname in df.columns]
    df = df.fillna(0)
    df = df.astype('int8')
    df = df.reset_index(['hadm_id'])
    
    
    return df



In [12]:
df_selected_labs = keep_last_labtest_instance(df_lab_events)

In [13]:
df_selected_labs.shape

(2403143, 5)

In [14]:
df_lab_results = pivot_labtests_to_columns(df_selected_labs)

In [15]:
lab_results_feature_names = df_lab_results.columns.tolist()

In [16]:
df_lab_results.shape

(53874, 410)

In [17]:
df_lab_results.head()

,hadm_id,50800,50801,50802,50803,50804,50805,50806,50808,50809,...,51513,51514,51515,51516,51517,51518,51519,51520,51523,51529
0,100001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NONE,NaN,NaN,NaN
1,100003,ART,NaN,-6.0,NaN,17.0,NaN,NaN,1.06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100006,ART,NaN,0.0,NaN,27.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NEG,NaN,NaN,NaN,NaN,NaN,NaN,HOLD,NaN
4,100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NONE,NaN,NaN,NaN


In [18]:
# clean categorical values
df_lab_results['51501'] = np.where(df_lab_results['51501'].isin(['<1','1','2']), '0-2',
                     np.where(df_lab_results['51501'].isin(['3','4']),'3-5', df_lab_results['51501']))

df_lab_results['51506'] = np.where(df_lab_results['51506'].isin(['CLEAR']), 'Clear',
                    np.where(df_lab_results['51506'].isin(['SLHAZY']), 'SlHazy',
                    np.where(df_lab_results['51506'].isin(['HAZY']), 'Hazy',
                    np.where(df_lab_results['51506'].isin(['SlCloudy']),'SlCldy',
                    np.where(df_lab_results['51506'].isin(['CLOUDY']),'Cloudy',df_lab_results['51506'])))))

df_lab_results['51463'] = np.where(df_lab_results['51463'].isin(['0']), 'NEG',
                    np.where(df_lab_results['51463'].isin(['NOTDONE']), 'NONE',
                    np.where(df_lab_results['51463'].isin(['LRG']), 'MANY', df_lab_results['51463'])))

df_lab_results['51508'] = np.where(df_lab_results['51508'].isin(['YELLOW','YEL']), 'Yellow',
                    np.where(df_lab_results['51508'].isin(['STRAW']), 'Straw',
                    np.where(df_lab_results['51508'].isin(['AMBER','AMB']), 'Amber',
                    np.where(df_lab_results['51508'].isin(['RED']), 'Red',
                    np.where(df_lab_results['51508'].isin(['ORANGE']), 'Orange',
                    np.where(df_lab_results['51508'].isin(['DKAMB','DKAMBER']), 'DkAmb',
                    np.where(df_lab_results['51508'].isin([' ']), np.nan, df_lab_results['51508'])))))))

# >80 is a category by iteslef, so keeping it. 
# df_lab_results['51484'] = np.where(df_lab_results['51484'].isin(['>80']), '80',df_lab_results['51484'])

# >300 is a category by itself, so keeping it
# df_lab_results['51492'] = np.where(df_lab_results['51492'].isin(['>300']), '300',
#                     np.where(df_lab_results['51492'].isin([' ']), np.nan, df_lab_results['51492']))
df_lab_results['51492'] = np.where(df_lab_results['51492'].isin([' ']), np.nan, df_lab_results['51492'])

df_lab_results['51514'] = np.where(df_lab_results['51514'].isin(['.2']), '0.2',
                    np.where(df_lab_results['51514'].isin(['>8']), '>8.0',
                    np.where(df_lab_results['51514'].isin(['>12']), '>12.0',
                    np.where(df_lab_results['51514'].isin(['NotDone',' ']), np.nan, df_lab_results['51514']))))

df_lab_results['51003'] = np.where(df_lab_results['51003'].isin({'<0.01','LESS THAN 0.01', '<0.010', 'LESS THAN 0.010','<0.10', '<0.02', 'LESS THAN 0.1' }), '0.001',
                    np.where(df_lab_results['51003'].isin({ 'GREATER THAN 25.0', '>25.0', '>25', 'GREATER THAN 25', '>25.00' }), '30.0',
                    df_lab_results['51003']))


df_lab_results['51519'] = np.where(df_lab_results['51519'].isin(['0', 'N']), 'NONE',
                    np.where(df_lab_results['51519'].isin(['NOTDONE']), np.nan, df_lab_results['51519']))

df_lab_results['51266'] = np.where(df_lab_results['51266'].isin(['UNABLE TO ESTIMATE DUE TO PLATELET CLUMPS']), 'NOTDETECTED', df_lab_results['51266'])

df_lab_results['51478'] = df_lab_results['51478'].map(
    {
        'Neg': 'NEG',
        'N': 'NEG',
    }
)

df_lab_results['51484'] = df_lab_results['51484'].map(
    {
        'T': 'TR',
        'Tr': 'TR',
        'Neg': 'NEG',
        '>80': '150.0'
    }
)
df_lab_results['51492'] = df_lab_results['51492'].map(
    {
        'Neg': 'NEG',
        '>600': '600.0',
        '>300': '500.0',
    }
)
df_lab_results['51463'] = df_lab_results['51463'].map(
    {
        ' ': 'NONE',
        'F': 'FEW',
        'MOD-': 'MOD',
        '1.0': 'RARE',
        '7I': 'NONE',
        '2.0': 'FEW'
    }
)
df_lab_results['51003'] = np.where(df_lab_results['51003'].isin({'NotDone', 'NOT DONE', 'ERROR', 'NOT DONE , TOTAL CK LESS THAN 100'}), np.nan,
                    np.where(df_lab_results['51003'].isin({ '>500', 'GREATER THAN 500' }), '600.0',
                             np.where(df_lab_results['51003'].isin({ '<1' }), '0.0', df_lab_results['51003'])))
df_lab_results['50922'] = np.where(df_lab_results['50922'].isin({'NEG', 'NEGATIVE', 'ERROR'}), -1.0, df_lab_results['50922'])

df_lab_results['51493'] = df_lab_results['51493'].map(
    {'0-2': '1.0',
     '3-5': '4.0',
     '11-20': '15.0',
     '>50': '80.0',
     '6-10': '8.0',
     '21-50': '35.0',
     '<1': '0.01',
     '>1000': '1100.0',
     'O-2': '1.0',
     ' 3-5': '4.0',
     ' ': np.nan,
     'LOADED': np.nan,
     ' 0-2': '1.0',
     'NOTDONE': np.nan,
     '0-20-2': np.nan,
     '0-2+' : np.nan,
     'TNTC' : np.nan,
     '3/5'  : np.nan,
     '21-200-2':  np.nan})

df_lab_results['51516'] = df_lab_results['51516'].map(
    {'0-2': '1.0',
     '3-5': '4.0',
     '11-20': '15.0',
     '>50': '80.0',
     '6-10': '8.0',
     '21-50': '35.0',
     '<1': '0.01',
     '>1000': '1100.0',
     'O-2': '1.0',
     ' 3-5': '4.0',
     ' ': np.nan,
     'LOADED': np.nan,
     ' 0-2': '1.0',
     'NOTDONE': np.nan,
     '0-20-2': np.nan,
     '0-2+' : np.nan,
     'TNTC' : np.nan,
     '3/5'  : np.nan,
     '21-200-2':  np.nan})


df_lab_results['51476'] = df_lab_results['51476'].map(
    {'0-2': '1.0',
     '3-5': '4.0',
     '11-20': '15.0',
     '>50': '80.0',
     '6-10': '8.0',
     '21-50': '35.0',
     '<1': '0.01',
     '>1000': '1100.0',
     'O-2': '1.0',
     ' 3-5': '4.0',
     ' ': np.nan,
     'LOADED': np.nan,
     ' 0-2': '1.0',
     'NOTDONE': np.nan,
     '0-20-2': np.nan,
     '0-2+' : np.nan,
     'TNTC' : np.nan,
     '3/5'  : np.nan,
     '21-200-2':  np.nan,
    '0-2,TRANS': '1.0',
    '<1 /HPF': '0.5',
     '11-20-': '15.0',
     '0.-2': '1.0',
     ' 0-2': '1.0',
    })

df_lab_results['50911'] = df_lab_results['50911'].map(
    {
 'NotDone': np.nan,
 '>500' : '600.0',
 'GREATER THAN 500' : '550.0',
 'NOT DONE' : np.nan,
 '<1' : '0.0',
 'ERROR': np.nan,
 'NOT DONE , TOTAL CK LESS THAN 100':  np.nan}
)

# dropping non interesting columns
df_lab_results = df_lab_results.drop(columns=['50827', '50856', '51100','51482', '50981'])

#### Drop sparse columns

In [19]:
df_lab_results = drop_sparse_columns(
    df_lab_results, 
    columns=  df_lab_results.drop(columns=['hadm_id']).columns.tolist(), 
    max_sparsity_to_keep=0.95
)

In [20]:
def set_columns_types(df):
    df = df.convert_dtypes(infer_objects=True)
    return df
    

In [21]:
def detect_data_types(df, columns=None):
    if columns is None:
        columns = df.columns.tolist()
    numeric = []
    categorical = []
    weird = []
    N = df.shape[0]
    for code in columns:
        n_missing = df_lab_results[code].isna().sum()
        size = N - n_missing
        size_unique = df_lab_results[code].nunique()
        sum_na = pd.to_numeric(df_lab_results[code][df_lab_results[code].notna()], errors='coerce').isna().sum()
        if sum_na / size < 0.05:
            numeric.append(code)
        elif sum_na / size > 0.05 and size_unique < 100:
            categorical.append(code)
        else:
            weird.append(code)
    return numeric, categorical, weird

def set_numeric_columns(df, numeric_columns: list):
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce', axis=1)
    return df

In [22]:
numeric, categorical, weird = detect_data_types(df_lab_results.drop(columns=['hadm_id']))

In [23]:
set_numeric_columns(df_lab_results, numeric)

,hadm_id,50800,50801,50802,50803,50804,50806,50808,50809,50810,...,51487,51491,51493,51498,51506,51508,51514,51516,51519,51523
0,100001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.5,NaN,1.008,NaN,NaN,NaN,NaN,NONE,NaN
1,100003,ART,NaN,-6.0,NaN,17.0,NaN,1.06,NaN,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100006,ART,NaN,0.0,NaN,27.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NEG,8.0,NaN,1.013,Clear,Yellow,NEG,NaN,NaN,HOLD
4,100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,1.014,NaN,NaN,NaN,NaN,NONE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53869,199993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53870,199994,ART,NaN,-3.0,NaN,27.0,NaN,1.32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53871,199995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53872,199998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NEG,6.5,NaN,1.010,Clear,Yellow,4.0,NaN,NaN,NaN


In [24]:
write_dataframe(df_lab_results, 'df_lab_results')

## @

In [107]:
# load prev results
df_lab_results = load_dataframe('df_lab_results')
lab_results_feature_names = df_lab_results.columns.tolist()

In [26]:
df_lab_flags = pivot_flags_to_columns(df_selected_labs)

In [27]:
lab_flags_feature_names = df_lab_flags.columns.tolist()

In [28]:
df_lab_flags.shape

(53874, 410)

In [29]:
df_lab_flags.head()

,hadm_id,50800_flag,50801_flag,50802_flag,50803_flag,50804_flag,50805_flag,50806_flag,50808_flag,50809_flag,...,51513_flag,51514_flag,51515_flag,51516_flag,51517_flag,51518_flag,51519_flag,51520_flag,51523_flag,51529_flag
0,100001,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,-1,0,0,0
1,100003,-1,0,-1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,100006,-1,0,-1,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,100007,0,0,0,0,0,0,0,0,0,...,0,-1,0,0,0,0,0,0,-1,0
4,100009,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,-1,0,0,0


In [30]:
write_dataframe(df_lab_flags, 'df_lab_flags')

## @

In [115]:
# load prev results
df_lab_flags = load_dataframe('df_lab_flags')
lab_flags_feature_names = df_lab_flags.columns.tolist()

## join to a single dataset

In [31]:
df_lab = df_lab_results.merge(df_lab_flags,how='left', on=['hadm_id'])

In [32]:
df_lab = df_lab.set_index('hadm_id').reindex(sorted(df_lab.columns), axis=1).drop(columns=['hadm_id']).reset_index()

In [33]:
df_lab.head()

,hadm_id,50800,50800_flag,50801,50801_flag,50802,50802_flag,50803,50803_flag,50804,...,51516,51516_flag,51517_flag,51518_flag,51519,51519_flag,51520_flag,51523,51523_flag,51529_flag
0,100001,None,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,-1,0,0,NONE,-1,0,None,0,0
1,100003,ART,-1,NaN,0,-6.0,-1,NaN,0,17.0,...,NaN,0,0,0,None,0,0,None,0,0
2,100006,ART,-1,NaN,0,0.0,-1,NaN,0,27.0,...,NaN,0,0,0,None,0,0,None,0,0
3,100007,None,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,0,0,0,None,0,0,HOLD,-1,0
4,100009,None,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,-1,0,0,NONE,-1,0,None,0,0


In [118]:
df_lab = df_lab.set_index(['hadm_id'])
df_static_data = df_static_data.set_index(['hadm_id'])

In [119]:
df_lab = df_lab.sort_values('hadm_id', axis=0)
df_static_data = df_static_data.sort_values('hadm_id', axis=0)

In [120]:
df_dataset_unprocessed = df_lab.join(df_static_data,how='inner') # join on index hadm_id

In [121]:
df_dataset_unprocessed_feature_names = df_dataset_unprocessed.columns.to_list()

In [122]:
df_dataset_unprocessed.head()

,50800_flag,50801_flag,50802_flag,50803_flag,50804_flag,50805_flag,50806_flag,50808_flag,50809_flag,50810_flag,...,51529_flag,admission_type,admission_location,insurance,language,religion,marital_status,ethnicity,gender,age
hadm_id,,,,,,,,,,,,,,,,,,,,,
100001,0,0,0,0,0,0,0,0,0,0,...,0,EMERGENCY,CLINIC REFERRAL/PREMATURE,Private,ENGL,PROTESTANT QUAKER,DIVORCED,WHITE,F,35.0
100003,-1,0,-1,0,1,0,0,1,0,-1,...,0,EMERGENCY,EMERGENCY ROOM ADMIT,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,M,59.0
100006,-1,0,-1,0,-1,0,0,0,0,0,...,0,EMERGENCY,EMERGENCY ROOM ADMIT,Private,OTHER,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,F,48.0
100007,0,0,0,0,0,0,0,0,0,0,...,0,EMERGENCY,EMERGENCY ROOM ADMIT,Private,OTHER,JEWISH,MARRIED,WHITE,F,73.0
100009,0,0,0,0,0,0,0,0,0,0,...,0,EMERGENCY,TRANSFER FROM MED FACILITY,Private,OTHER,CATHOLIC,MARRIED,WHITE,M,60.0


In [123]:
df_dataset_unprocessed.shape

(53874, 418)

In [124]:
write_dataframe(df_dataset_unprocessed, 'df_dataset_unprocessed')

### @

In [23]:
# load prev results
df_dataset_unprocessed = load_dataframe('df_dataset_unprocessed')
df_dataset_unprocessed_feature_names = df_dataset_unprocessed.columns.tolist()

### numeric values: clean and standardize 

In [125]:
def stanardize_numeric_values(df, list_of_clms=None, replace_missing=False, ):
    """
    Use the median and interquartile range to 
    standardize the numeric variables
    value = (value – median) / (p75 – p25)
    """
    if list_of_clms is None:
        list_of_clms = df_stats.columns.tolist()
    df_stats = df[list_of_clms].describe(percentiles=[.01,.25, .5, .75, .95, .99])
    list_of_clms = df_stats.columns.tolist()
    
    for code in list_of_clms:
        median = df_stats[code]['50%']
        p25 = df_stats[code]['1%']
        p75 = df_stats[code]['99%']
        df[code] = (df[code] - median) / (p75 - p25)
    return df

from sklearn.impute import SimpleImputer

def replace_missing_val(df, list_of_clms, how='median'):
    """
    Imputation of missing values using median
    """
    temp_df = df[list_of_clms]
    imp = SimpleImputer(strategy=how)
    df_prc = imp.fit_transform(temp_df)
    temp_df = pd.DataFrame(df_prc, columns=list_of_clms, index=df.index)
    return temp_df

In [126]:
columns_to_standardize = [col for col in df_dataset_unprocessed.columns.tolist() if not col.endswith('_flag')]
df_dataset_unprocessed = stanardize_numeric_values(df_dataset_unprocessed, columns_to_standardize)

In [127]:
numeric_cols = df_dataset_unprocessed.select_dtypes('number')
numeric_cols = [col for col in numeric_cols if not col.endswith('_flag')]
# df_dataset_unprocessed = replace_missing_val(df_dataset_unprocessed, numeric_cols, how='median')

In [128]:
# df_dataset_unprocessed = 

df_new_numeric = replace_missing_val(df_dataset_unprocessed, numeric_cols )

In [129]:
df_dataset_unprocessed = df_dataset_unprocessed.drop(columns=numeric_cols).join(df_new_numeric)

### categorical values: One Hot Encode

In [130]:
categorical_cols = df_dataset_unprocessed.select_dtypes('object').columns.tolist()

In [131]:
df_dataset_processed = pd.get_dummies(df_dataset_unprocessed, columns=categorical_cols, dummy_na=True, drop_first=True)

In [132]:
df_dataset_processed.fillna(0)

,50800_flag,50801_flag,50802_flag,50803_flag,50804_flag,50805_flag,50806_flag,50808_flag,50809_flag,50810_flag,...,ethnicity_HISPANIC/LATINO - GUATEMALAN,ethnicity_HISPANIC/LATINO - HONDURAN,ethnicity_OTHER,ethnicity_SOUTH AMERICAN,ethnicity_UNKNOWN/NOT SPECIFIED,ethnicity_WHITE,ethnicity_WHITE - EASTERN EUROPEAN,ethnicity_nan,gender_M,gender_nan
hadm_id,,,,,,,,,,,,,,,,,,,,,
100001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
100003,-1,0,-1,0,1,0,0,1,0,-1,...,0,0,0,0,0,1,0,0,1,0
100006,-1,0,-1,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
100009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
199994,-1,0,-1,0,-1,0,0,-1,0,0,...,0,0,0,0,0,1,0,0,0,0
199995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [133]:
df_dataset_processed.head()

,50800_flag,50801_flag,50802_flag,50803_flag,50804_flag,50805_flag,50806_flag,50808_flag,50809_flag,50810_flag,...,ethnicity_HISPANIC/LATINO - GUATEMALAN,ethnicity_HISPANIC/LATINO - HONDURAN,ethnicity_OTHER,ethnicity_SOUTH AMERICAN,ethnicity_UNKNOWN/NOT SPECIFIED,ethnicity_WHITE,ethnicity_WHITE - EASTERN EUROPEAN,ethnicity_nan,gender_M,gender_nan
hadm_id,,,,,,,,,,,,,,,,,,,,,
100001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
100003,-1,0,-1,0,1,0,0,1,0,-1,...,0,0,0,0,0,1,0,0,1,0
100006,-1,0,-1,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
100009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [134]:
write_dataframe(df_dataset_processed, 'df_dataset_processed')

## @

In [146]:
df_dataset_processed = load_dataframe('df_dataset_processed')

In [135]:
df_features = df_dataset_processed

## Build Dataset for training AutoEncoder


In [154]:
df_features['y']=np.zeros((df_features.shape[0],))

In [157]:
fulldata = df_features.to_numpy()
# Save to a file
np.save('data/fulldata.npy', fulldata)


## Load Patients cohort
hadm_id, 
class label

In [58]:
df_cohort = cohort.query_esbl_pts(params.observation_window_hours)
df_cohort = cohort.remove_dups(df_cohort)
df_cohort = df_cohort[['hadm_id', 'RESISTANT_YN']]

In [59]:
df_cohort.shape

(4719, 2)

In [230]:
write_dataframe(df_cohort, 'df_cohort')

## @

In [136]:
df_cohort = load_dataframe('df_cohort')

Join the cohort on the features

In [137]:
df_full_data = df_cohort.set_index(['hadm_id']).join(df_features.reset_index().set_index(['hadm_id']), how='inner')


In [138]:
df_full_data = df_full_data.rename(columns={'RESISTANT_YN': 'y'})

In [139]:
y_col =  df_full_data['y']
df_full_data = df_full_data.drop(columns=['y'])
df_full_data['y']=y_col
df_full_data = df_full_data.reset_index(drop=True)

In [140]:
write_dataframe(df_full_data, 'df_full_data')

## @

In [144]:
df_full_data = load_dataframe('df_full_data')

In [145]:
df_full_data

,50800_flag,50801_flag,50802_flag,50803_flag,50804_flag,50805_flag,50806_flag,50808_flag,50809_flag,50810_flag,...,ethnicity_HISPANIC/LATINO - HONDURAN,ethnicity_OTHER,ethnicity_SOUTH AMERICAN,ethnicity_UNKNOWN/NOT SPECIFIED,ethnicity_WHITE,ethnicity_WHITE - EASTERN EUROPEAN,ethnicity_nan,gender_M,gender_nan,y
0,-1,0,-1,0,-1,-1,1,-1,1,-1,...,0,0,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,-1,0,-1,0,-1,0,0,-1,1,-1,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,-1,-1,-1,-1,-1,0,-1,-1,1,-1,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407,0,0,-1,0,-1,0,0,-1,0,-1,...,0,0,0,0,1,0,0,1,0,0
4408,0,-1,-1,0,-1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4409,-1,0,-1,0,1,0,-1,1,1,-1,...,0,0,0,1,0,0,0,1,0,0
4410,0,0,-1,0,-1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


## export to numpy

In [142]:
fulldata = df_full_data.to_numpy()

In [143]:
# Save to a file
np.save('data/fulldata.npy', fulldata)

In [279]:
# Load data
fulldata = np.load('data/fulldata.npy')

In [74]:
fulldata.shape

(4412, 720)

In [278]:
# Load data
labdata = np.load('data/labdata.npy')

In [281]:
labdata

array([[ 0.0475698 ,  0.0212766 , -0.5       , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.114788  ,  0.12765957, -0.5       , ...,  1.        ,
         0.        ,  0.        ],
       [-0.05274043,  0.        , -0.5       , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.04550155,  0.        , -0.5       , ...,  1.        ,
         0.        ,  0.        ],
       [-0.00620476, -0.0212766 , -0.5       , ...,  0.        ,
         1.        ,  0.        ],
       [-0.00620476, -0.0212766 , -0.5       , ...,  1.        ,
         0.        ,  0.        ]])

## Test on a simple model

before cleaning the dataset, check with an XGBoost that can handle missing data.

In [67]:
import torch
import numpy as np
from sklearn.metrics import make_scorer, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_validate, learning_curve
from torch.utils.data import Dataset
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

params.validation_set_fraction=0.29
params.test_set_fraction=0.01
params.negative_to_positive_ratio=2
print(params.__dict__)
df_X = df_full_data.drop(columns = ['y'] )
df_y = df_full_data[['y']]
X_train, X_validate, y_train, y_validate = train_test_split(df_X, df_y, stratify=df_y, test_size=0.29, random_state=params.random_state)
print(f"Train count: {Counter(y_train['y'])}")
print(f"Validate count: {Counter(y_validate['y'])}")
# X_train = train_set.drop(columns=['y'])
# y_train = train_set['y'].astype('int')
# X_validate = df_validation.drop(columns=['y'])
# y_validate = df_validation['y'].astype('int')
print(np.mean(y_validate['y']))
print(f"Train shape: {X_train.shape}")
print(f"Validate shape: {X_validate.shape}")


{'observation_window_hours': 6, 'antibiotics_name': ['CEFTAZIDIME'], 'bacteria_ids': [80004, 80026, 80005, 80017, 80040, 80008, 80007, 80002], 'negative_to_positive_ratio': 2, 'test_set_fraction': 0.01, 'validation_set_fraction': 0.29, 'train_set_fraction': 0.8, 'random_state': 11}
Train count: Counter({0: 2598, 1: 534})
Validate count: Counter({0: 1062, 1: 218})
0.1703125
Train shape: (3132, 719)
Validate shape: (1280, 719)


In [68]:
def print_accuracy(cv_scores):
    # print("Accuracy: %0.2f (+/- %0.2f)" % (cv_scores.mean(), cv_scores.std() * 2))
    for score_name, scores in cv_scores.items():
        print("%s: %0.2f (+/- %0.2f)" % (score_name, scores.mean(), scores.std() * 2))
        
def train_xgboost():
    import xgboost as xgb

    param_dist = dict(objective='binary:logistic',
                      n_estimators=100, # 170,
                      eval_metric='rmsle', # 'logloss',
                      max_depth=4,
                      eta=0.3,
                      booster='gbtree',
                      n_jobs=4,
#                       enable_categorical=True
                      # subsample=0.8,
                      # colsample_bynode=0.5
                    )

    xgboost_cls = xgb.XGBClassifier(**param_dist)
    xgboost_cls.fit(X_train, y_train)
    return xgboost_cls


# model = train_random_forest()
model = train_xgboost()
y_validate_hat = model.predict(X_validate)
print(f"predictions mean: {np.mean(y_validate_hat)}")
simple_score = model.score(X_validate, y_validate)
print(f"simple_score: {simple_score}")
# scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy', 'Precision': 'precision', 'Recall': 'recall'}

scoring = ['roc_auc','accuracy','precision', 'recall', 'f1']
cv_scores = cross_validate(model, X_train, y_train['y'], scoring=scoring)
print_accuracy(cv_scores)
print(classification_report(y_validate, y_validate_hat))

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generat

predictions mean: 0.04609375
simple_score: 0.81796875


/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(lab

fit_time: 4.63 (+/- 0.15)
score_time: 0.03 (+/- 0.01)
test_roc_auc: 0.67 (+/- 0.04)
test_accuracy: 0.81 (+/- 0.01)
test_precision: 0.33 (+/- 0.10)
test_recall: 0.12 (+/- 0.04)
test_f1: 0.18 (+/- 0.05)
              precision    recall  f1-score   support

           0       0.84      0.97      0.90      1062
           1       0.37      0.10      0.16       218

    accuracy                           0.82      1280
   macro avg       0.61      0.53      0.53      1280
weighted avg       0.76      0.82      0.77      1280



/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")


Import cohort/labels data from the .pkl file:

## Tools to explore too many unique values

In [ ]:
df_dataset_unprocessed[categorical_cols].nunique().sort_values(ascending=False)[10:20]

In [ ]:
allvals = df_dataset_unprocessed['50911'].unique().tolist()
len(allvals), df_dataset_unprocessed['50911'].notna().sum()

In [ ]:
df_dataset_unprocessed['50911'].value_counts()

In [ ]:
[x for x in allvals if x and not x == np.nan and not str(x).replace('.','').isnumeric()]

In [ ]:
sorted([float(x) for x in allvals if x and str(x).replace('.','').isnumeric()])


In [ ]:
df_dataset_unprocessed.nunique().sort_values(ascending=False)

## THE END

In [159]:
import torch
torch.__version__

'1.5.1'

In [46]:
fulldata

array([[ 0.0475698 ,  0.0212766 , -0.5       , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.114788  ,  0.12765957, -0.5       , ...,  1.        ,
         0.        ,  0.        ],
       [-0.05274043,  0.        , -0.5       , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.04550155,  0.        , -0.5       , ...,  1.        ,
         0.        ,  0.        ],
       [-0.00620476, -0.0212766 , -0.5       , ...,  0.        ,
         1.        ,  0.        ],
       [-0.00620476, -0.0212766 , -0.5       , ...,  1.        ,
         0.        ,  0.        ]])